In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sf-crime/sampleSubmission.csv
/kaggle/input/sf-crime/test.csv.zip
/kaggle/input/sf-crime/train.csv.zip
/kaggle/input/sf-crime/train.csv
/kaggle/input/sf-crime/test.csv
/kaggle/input/sf-crime/sf_map_copyright_openstreetmap_contributors.txt
/kaggle/input/sf-crime/sf_map_copyright_openstreetmap_contributors.rds


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

train = pd.read_csv('/kaggle/input/sf-crime/train.csv', parse_dates=['Dates'])
test = pd.read_csv('/kaggle/input/sf-crime/test.csv', parse_dates=['Dates'], index_col='Id')

from keras.preprocessing.text import Tokenizer
max_words = 8000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(train["Address"]) + list(test["Address"]))

haha = tokenizer.texts_to_sequences(train["Address"])
haha2 = tokenizer.texts_to_sequences(test["Address"])

from keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(haha, maxlen=6)
padded2 = pad_sequences(haha2, maxlen=6)

from keras import Sequential
from keras.layers import Flatten, Embedding, Dense 
model = Sequential()
model.add(Embedding(padded.shape[0], 1, input_length=6))
model.add(Flatten())
model.add(Dense(39, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])
model.fit(padded, y, epochs = 10 , batch_size = 2048)

preds = model.predict(padded)
preds2 = model.predict(padded2)

from sklearn.decomposition import PCA

pca1 = PCA(n_components=2)
X_low = pca1.fit_transform(preds)
#X2 = pca1.inverse_transform(X_low)
X_low2 = pca1.transform(preds2)

train['Date'] = pd.to_datetime(train['Dates'].dt.date)
train['n_days'] = (train['Date'] - train['Date'].min()).apply(lambda x: x.days)
train['Day'] = train['Dates'].dt.day
train['DayOfWeek'] = train['Dates'].dt.weekday
train['Month'] = train['Dates'].dt.month
train['Year'] = train['Dates'].dt.year
train['Hour'] = train['Dates'].dt.hour
train['Minute'] = train['Dates'].dt.minute
train['Block'] = train['Address'].str.contains('block', case=False).apply(lambda x: 1 if x == True else 0)
train['ST'] = train['Address'].str.contains('ST', case=False).apply(lambda x: 1 if x == True else 0)

test['Date'] = pd.to_datetime(test['Dates'].dt.date)
test['n_days'] = (test['Date'] - test['Date'].min()).apply(lambda x: x.days)
test['Day'] = test['Dates'].dt.day
test['DayOfWeek'] = test['Dates'].dt.weekday
test['Month'] = test['Dates'].dt.month
test['Year'] = test['Dates'].dt.year
test['Hour'] = test['Dates'].dt.hour
test['Minute'] = test['Dates'].dt.minute
test['Block'] = test['Address'].str.contains('block', case=False).apply(lambda x: 1 if x == True else 0)
test['ST'] = test['Address'].str.contains('ST', case=False).apply(lambda x: 1 if x == True else 0)

from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])

train = pd.concat([train, pd.DataFrame(X_low)], 1)
test = pd.concat([test, pd.DataFrame(X_low2)], 1)

le3 = LabelEncoder()
le3.fit(list(train['Address']) + list(test['Address']))
train['Address'] = le3.transform(train['Address'])
test['Address'] = le3.transform(test['Address'])

train.drop(['Dates','Date','Descript','Resolution', 'Category'], 1, inplace=True)
test.drop(['Dates','Date',], 1, inplace=True)

from lightgbm import LGBMClassifier

#hyper = {'colsample_bytree': 0.6254687581749554,
# 'is_unbalance': False,
# 'learning_rate': 0.10240007357745745,
# 'min_child_samples': 105,
# 'num_class': 39,
# 'num_leaves': 233,
# 'objective': 'multiclass',
# 'reg_alpha': 0.4000134592012641,
# 'reg_lambda': 0.5082596745249518,
# 'subsample': 0.9338693244190213,
# 'subsample_for_bin': 140000,
# 'n_estimators': 150}
model = LGBMClassifier(num_leaves: 233,n_estimators=150,learning_rate=0.1)
model.fit(train, y, categorical_feature=["PdDistrict", "DayOfWeek"])
preds = model.predict_proba(test)
submission = pd.DataFrame(preds, columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')), index=test.index)
submission.to_csv('LGBM_final.csv', index_label='Id')

SyntaxError: invalid syntax (<ipython-input-2-efde1586ba7a>, line 102)